# Visual SLAM Trilogy
## Part III: loop Closure

In this SLAM hands-on lecture, we will implement a visual SLAM system that has an OpenCV frontend, a GTSAM backend, and a loop closure module based on a bag-of-words approach.

## Overview

The overview of our SLAM system is depicted below, simplified by certain assumptions:

1. Odometry
    - Assumes there is an odometry trajectory provided to our SLAM system.
    - In practice, this includes a Kalman filter fusing the IMU and encoder data.
2. Frontend:
    - Processes the raw sensor data and extracts relevant features for optimization.
    - Associates each measurement to a specific landmark (3D point).
    - Provide initial values for the backend variables.
3. Mapping
    - Utilizes a very minimum sparse map.
    - Could be replaced with OGM or even 3D Gaussian Splatting in the future.
4. Backend
    - Solve the maximum a posteriori (MAP) estimation problem.
    - Feed back information to loop closure.
5. Loop closure:
    - Acts as a long-term tracking module (compared to the short-term tracking module in frontend).
    - Implemented with visual bag-of-word algorithm.

![slam_overview](assets/slam_overview.png)

## Dataset

We will use the abandoned_factory P006 sequence from the TartanAir dataset to test the system. It is a simulation dataset with diverse environments and ground truth dataset, which make it perfect for testing and evaluating our system. To get started, we'll need to access the camera intrinsics, extrinsics, and data format information, which can be found here: https://github.com/castacks/tartanair_tools/blob/master/data_type.md.

## Implementation

In this notebook, we will explore the concept of loop closure by building a toy example. Specifically, we will cover the following topics:

- Detect the loop by calculating the confusion matrix using Bag of Word (BoW) algorithm
- Optimize the trajectory with GTSAM
- Future readings about different categories of visual SLAMs

## I. Dependency


### 1. Install Python libraries

In [ ]:
# install the minslam package in “editable” mode
# !pip install -e ..

# install other libraries
# !pip install numpy spatialmath-python opencv-python matplotlib gtsam ipympl evo plotly

### 2. Import libraries and dataset
Please download [abadoned_factory P006 dataset](https://drive.google.com/file/d/1Q_fSI0U-IMfv90lyE1Uh78KV2QJheHbv/view?usp=share_link) and extract it to a folder named "data".

In [ ]:
# this block should run without error

# dataset
import os

# change the perspective of 3d plots with this command
%matplotlib widget

# test if we can find the dataset
dataset_folder = '/home/junzhewu/Projects/minimum-slam/data/tartanair/scenes/abandonedfactory/Easy/P006'
print('Check if the data folder exists:',os.path.exists(dataset_folder))

# visualization
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# frontend
import numpy as np
from spatialmath import *

# backend
import gtsam
from gtsam.symbol_shorthand import L, X

# our slam implementation
from minslam.data_loader import TartanAirLoader, plot_trajectory
from minslam.frontend import Frontend
from minslam.params import Params
from minslam.backend import Backend
from minslam.camera import PinholeCamera

## II. Loop Closure

### 0. Detect Loop

We will be using the visual bag of word method, similar to ORB-SLAM2, to detect a loop in the trajectory. However, since there are no well-built visual bag of word libraries in Python, we will use C++ to call visual bag of word and compute the confusion matrix, and then visualize the confusion matrix in this notebook. For the C++ loop closure detection implementation, you can refer to [tccoin/simple_slam_loop_closure](https://github.com/tccoin/simple_slam_loop_closure).

Once we obtain the confusion matrix from the C++ algorithm, we should visualize it. Each pixel in the matrix represents the similarity between frames i and j. Higher pixel values indicate a greater likelihood of a loop closure.

In [ ]:
confusion = np.loadtxt('confusion_tartanair_af_P006.txt', delimiter=',')
frame_interval = 1
N = confusion.shape[0]
# plot
fig = px.imshow(confusion, color_continuous_scale='gray')
fig.update_layout(title='Confusion Matrix', width=800, height=800)
fig.show()

The values along the diagonal of the matrix are significantly high, which is expected as adjacent frames typically appear similar. This information is already leveraged in the local pose estimation phase, where feature tracking helps optimize landmark positions and poses. However, the global loop closures we aim to detect should involve non-adjacent frames. Therefore, we will only consider loop closure candidates that are separated by at least 100 frames to ensure they are truly global.

In [ ]:
min_loop_interval = 100
confusion_filtered = confusion.copy()
for i in range(N):
    confusion_filtered[i:i+int(min_loop_interval/frame_interval), i] = 0
# plot
fig = px.imshow(confusion_filtered, color_continuous_scale='gray')
fig.update_layout(title='Filtered Confusion Matrix', width=800, height=800)
fig.show()

Next, we will evaluate each loop closure candidate by identifying visual feature matches between two frames.

This step could be omitted if we enhance the accuracy of the bag of words algorithm. For instance, refining the vocabulary file by training it on the TartanAir dataset could lead to improvements.

In [ ]:
# for each frame, add a new candidate if the max confidence is above 0.015
loop_candidates = []
for i in range(N):
    j = np.argmax(confusion_filtered[i])
    if confusion_filtered[i, j] > 0.015:
        loop_candidates.append((i,j))
print('number of loop candidates:', len(loop_candidates))

# load dataset
dataset = TartanAirLoader(dataset_folder)
dataset.load_ground_truth()
gt_traj = np.array([x.t for x in dataset.gt])

# check visual similarity
params = Params('../params/tartanair.yaml')
frontend = Frontend(params)
matches_number = []
for i,j in loop_candidates:
    # calculate the number of matches for each candidate
    dataset.set_curr_index(i)
    pose = dataset.read_current_ground_truth()
    color, depth = dataset.read_current_rgbd()
    frontend.add_keyframe(pose, color, depth, dataset.curr_index)
    frontend.extract_features()
    dataset.set_curr_index(j)
    pose = dataset.read_current_ground_truth()
    color, depth = dataset.read_current_rgbd()
    frontend.add_keyframe(pose, color, depth, dataset.curr_index)
    frontend.extract_features()
    frontend.match_features()
    matches_number.append(len(frontend.curr_frame.matches))
# choose the candidate with the most matches
max_matches_idx = np.argmax(matches_number)
print('candidate with the most matches:', loop_candidates[max_matches_idx])
print('number of matches:', matches_number[max_matches_idx])

### 1. The loop

First, we will visualize the trajectory which includes a loop. It's important to note that a loop doesn't necessarily mean the robot comes back to the exact same place, but rather it should be near a place it visited before so that a relative pose between two frames can be found.

In [ ]:
# load seq P006
dataset = TartanAirLoader(dataset_folder)
dataset.load_ground_truth()
gt_traj = np.array([x.t for x in dataset.gt])

# set the start and end frame for the sequence
seq_start = 185
seq_end = 519
n_frames = seq_end - seq_start

# plot the loop
fig = make_subplots(rows=1, cols=1, specs=[[{'type':'scatter3d'}]])
fig.add_trace(go.Scatter3d(x=gt_traj[seq_start:seq_end,0], y=gt_traj[seq_start:seq_end,1], z=gt_traj[seq_start:seq_end,2], mode='lines', name='gt'))
fig.update_layout(scene=dict(aspectmode='cube'))
fig.show()


The start and end frame of the loop should have a similar viewpoint and share a significant number of visual features:

In [ ]:
dataset.set_curr_index(seq_start)
color_start, _ = dataset.read_current_rgbd()
dataset.set_curr_index(seq_end)
color_end, _ = dataset.read_current_rgbd()
image = np.hstack([color_start, color_end])

# plot using plotly
fig = make_subplots(rows=1, cols=1, specs=[[{'type':'image'}]])
fig.add_trace(go.Image(z=image))
fig.update_layout(scene=dict(aspectmode='cube'))
fig.show()

### 2. The drift

In practice, the odometry estimates can contain noise and drift due to various factors such as sensor noise and model inaccuracies. To simulate such scenarios, we can add some noise to the ground truth odometry and use it as the noisy odometry. In the figure below, we visualize both the ground truth and noisy odometry trajectories. Before we add loop closure, the estimated trajectory from GTSAM will be identical to the noisy odometry trajectory. It is important to note that due to the accumulated error in each frame, the start and end positions of the loop may differ significantly from the ground truth.

In [ ]:
gt_poses = dataset.gt
noisy_poses = dataset.add_noise(gt_poses[seq_start:], [3e-4, 3e-4], [1e-3, 3e-4], seed=530)
odom_poses = SE3([*gt_poses[:seq_start], *noisy_poses])
dataset.set_odometry(odom_poses)
dataset.set_ground_truth(gt_poses)

# for plotting
gt_traj = np.zeros((n_frames, 3))
odom_traj = np.zeros((n_frames, 3))

# add the odometry poses to the graph
for i in range(seq_start, seq_end):
    dataset.set_curr_index(i)
    gt_traj[i-seq_start] = dataset.read_current_ground_truth().t
    odom_traj[i-seq_start] = dataset.read_current_odometry().t

# plot the trajectories
fig = make_subplots(rows=1, cols=1, specs=[[{'type':'scatter3d'}]])
fig.add_trace(go.Scatter3d(x=odom_traj[:,0], y=odom_traj[:,1], z=odom_traj[:,2], mode='lines', name='odom'))
fig.add_trace(go.Scatter3d(x=gt_traj[:,0], y=gt_traj[:,1], z=gt_traj[:,2], mode='lines', name='gt'))
fig.update_layout(scene=dict(aspectmode='cube'))
fig.show()

### 4. Loop Closure Optimization

The goal is to build a pose graph and conduct the global pose optimization. To simplify the calculation, we will not add the visual landmarks.

To add the loop closure constraint, we first need to detect the loop, which is usually done using a visual bag-of-words approach. This involves extracting features from each frame and then creating a dictionary of visual words to represent them. Then, we can match the visual words between frames to create a graph of similar frames, where each node represents a frame and edges represent similar frames.

Once we have detected the loop and found the relative pose between the start and end frame using algorithm like [PnP](https://docs.opencv.org/4.x/d5/d1f/calib3d_solvePnP.html), we can add a loop closure constraint to the pose graph optimization. As a result, the estimated trajectory will be much closer to the ground truth.

In [ ]:
def to_gtsam_pose(pose):
    return gtsam.Pose3(gtsam.Rot3(pose.R), gtsam.Point3(pose.t))

# initialize the backend
graph = gtsam.NonlinearFactorGraph()
initial_estimate = gtsam.Values()

# add prior for the first pose
graph.push_back(gtsam.PriorFactorPose3(
    X(seq_start), to_gtsam_pose(odom_poses[seq_start]), gtsam.noiseModel.Diagonal.Sigmas(np.ones(6)*0.0001)
))
initial_estimate.insert(X(seq_start), to_gtsam_pose(odom_poses[seq_start]))

# for plotting
gt_traj = np.zeros((n_frames, 3))
odom_traj = np.zeros((n_frames, 3))

# add the odometry poses to the graph
for i in range(seq_start, seq_end):
    dataset.set_curr_index(i)
    gt_traj[i-seq_start] = dataset.read_current_ground_truth().t
    odom_traj[i-seq_start] = dataset.read_current_odometry().t
    initial_estimate.insert(X(i+1), to_gtsam_pose(odom_poses[i]))
    graph.push_back(gtsam.BetweenFactorPose3(
        X(i), X(i+1), to_gtsam_pose(odom_poses[i].inv()*odom_poses[i+1]),
        gtsam.noiseModel.Diagonal.Sigmas(np.ones(6)*0.5) # we set a higher noise for odometry
    ))

# add loop closures
graph.push_back(gtsam.BetweenFactorPose3(
    X(seq_end), X(seq_start), to_gtsam_pose(gt_poses[seq_end].inv()*gt_poses[seq_start]),
    gtsam.noiseModel.Diagonal.Sigmas(np.ones(6)*0.01)  # we set a lower noise for loop closure constraints
))
# TODO: try to set it to a higher value and see what happens

# optimize the graph
print('before optimizaiton error: ', graph.error(initial_estimate))
optimizer_params = gtsam.LevenbergMarquardtParams()
optimizer_params = gtsam.LevenbergMarquardtParams.CeresDefaults()
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate, optimizer_params)
current_estimate = optimizer.optimize()
estimated_traj = gtsam.utilities.extractPose3(current_estimate)[:, -3:]
print('after optimizaiton error: ', graph.error(current_estimate))


# plot the results
fig = make_subplots(rows=1, cols=1, specs=[[{'type':'scatter3d'}]])
fig.add_trace(go.Scatter3d(x=odom_traj[:,0], y=odom_traj[:,1], z=odom_traj[:,2], mode='lines', name='odom'))
fig.add_trace(go.Scatter3d(x=gt_traj[:,0], y=gt_traj[:,1], z=gt_traj[:,2], mode='lines', name='gt'))
fig.add_trace(go.Scatter3d(x=estimated_traj[:,0], y=estimated_traj[:,1], z=estimated_traj[:,2], mode='lines', name='estimated'))
fig.update_layout(scene=dict(aspectmode='cube'))
fig.show()

## III. Visual SLAM Categories

- **Feature-Based SLAM** detects and tracks distinct features within the environment.
    - Example: [ORB-SLAM 3](https://github.com/UZ-SLAMLab/ORB_SLAM3)

- **Direct SLAM** directly uses the intensity values of all pixels in an image to estimate the camera motion and structure of the environment. This method is usually more sensitive to lighting variations but can work well in low-texture environments where feature-based method struggles.
    - Example: [LSD-SLAM](https://github.com/tum-vision/lsd_slam)

- **Dense SLAM** creates a dense reconstruction of the environment using either stereo cameras or RGBD cameras. This method involves processing a larger amount of data to produce a more detailed map.
    - Example: [KinectFusion](https://github.com/ParikaGoel/KinectFusion), [ElasticFusion](https://github.com/mp3guy/ElasticFusion)

- **Semantic SLAM** incorporates machine learning techniques to understand and label different parts of the environment based on their meaning.
    - Example: [Khronos](https://github.com/MIT-SPARK/Khronos)

## IV. Conclusion
Congratulations! You have gained a solid understanding of the basic concepts and techniques used in real SLAM systems. By following the steps and examples we provided, you now have the knowledge to implement your own SLAM system in Python. Keep in mind that SLAM is a complex and evolving field, and there is always more to learn and explore. Don't hesitate to continue your learning journey and discover more advanced techniques and applications in SLAM.